In [6]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import tkinter as tk
from tkinter import filedialog, messagebox

# MPANET 모델 로드
model = load_model('C:/Users/public.DESKTOP-KSOI6C0/project/MPANET.h5')

# 감정 레이블
emotion_labels = ['anger', 'happiness', 'panic', 'sadness']

In [7]:
def preprocess_frame(frame, target_size=(224, 224)):
    frame = cv2.resize(frame, target_size)
    frame = frame.astype('float32') / 255.0
    frame = np.expand_dims(frame, axis=0)
    return frame

def predict_emotion(frame, model):
    processed_frame = preprocess_frame(frame)
    predictions = model.predict(processed_frame)
    return predictions

def get_frames(video_path, interval=0.25):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * interval)
    frames = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            frames.append(frame)
        frame_count += 1

    cap.release()
    return frames

def analyze_video():
    global emotion_score
    cap = cv2.VideoCapture('output.avi')
    frame_interval = int(cap.get(cv2.CAP_PROP_FPS) * 0.5)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            predictions = predict_emotion_video(frame)
            max_index = np.argmax(predictions)
            confidence = predictions[0][max_index]
            emotion = emotion_labels[max_index]

            # Apply different thresholds for each emotion
            if (emotion == 'anger' and confidence >= 0.95) or \
               (emotion == 'happiness' and confidence >= 0.50) or \
               (emotion == 'panic' and confidence >= 0.65) or \
               (emotion == 'sadness' and confidence >= 0.75):
                update_emotion_score(emotion)
        frame_count += 1

    cap.release()
    update_video_counts()
    update_score()

# The rest of your code remains unchanged



class EmotionAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Realtime Emotion Analysis")
        self.root.configure(bg='white')

        self.label = tk.Label(root, text="캠을 이용한 실시간 감지 혹은 분석하고 싶은 영상을 넣어주세요", bg='white')
        self.label.pack(pady=20, padx=20)

        self.start_button = tk.Button(root, text="실시간 감지 시작", command=self.start_webcam, bg='white')
        self.start_button.pack(pady=10, padx=20)

        self.stop_button = tk.Button(root, text="실시간 감지 중지", command=self.stop_webcam, bg='white')
        self.stop_button.pack(pady=10, padx=20)

        self.select_button = tk.Button(root, text="분석 영상 선택", command=self.select_video, bg='white')
        self.select_button.pack(pady=10, padx=20)

        self.show_result_window_button = tk.Button(root, text="투명 배경으로 분석창 띄우기", command=self.create_result_window, bg='white')
        self.show_result_window_button.pack(pady=10, padx=20)

        self.result_text = tk.Text(root, height=10, width=50, bg='black', fg='white')
        self.result_text.pack(pady=20, padx=20)

        self.cap = None
        self.running = False
        self.emotion_counts = {label: 0 for label in emotion_labels}

        self.result_window = None
        self.result_text_toplevel = None

    def select_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mov")])
        if file_path:
            self.label.config(text="비디오 분석중")
            self.root.update()
            emotion_counts = analyze_video(file_path)
            self.display_results(emotion_counts)
            self.update_result_window(emotion_counts)

    def start_webcam(self):
        self.cap = cv2.VideoCapture(0)
        self.running = True
        self.emotion_counts = {label: 0 for label in emotion_labels}
        self.result_text.delete(1.0, tk.END)
        self.update_webcam()

    def stop_webcam(self):
        self.running = False
        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()
        if self.result_window:
            self.result_window.destroy()
            self.result_window = None

    def create_result_window(self):
        if self.result_window is None:
            self.result_window = tk.Toplevel(self.root)
            self.result_window.attributes('-alpha', 0.5)
            self.result_window.attributes('-topmost', True)
            self.result_window.geometry("300x200")
            self.result_window.overrideredirect(True)
            self.result_window.configure(bg='black')  # Set entire window background to black
            self.result_text_toplevel = tk.Text(self.result_window, height=10, width=30, bg='black', fg='white', bd=0, highlightthickness=0)
            self.result_text_toplevel.pack()

            self.result_window.bind("<Button-1>", self.start_move)
            self.result_window.bind("<B1-Motion>", self.do_move)

    def start_move(self, event):
        self.x = event.x
        self.y = event.y

    def do_move(self, event):
        x = event.x_root - self.x
        y = event.y_root - self.y
        self.result_window.geometry(f"+{x}+{y}")

    def update_webcam(self):
        if self.running:
            ret, frame = self.cap.read()
            if ret:
                predictions = predict_emotion(frame, model)
                for i, label in enumerate(emotion_labels):
                    prob = predictions[0][i]
                    if (label == 'anger' and prob >= 0.90) or \
                       (label == 'happiness' and prob >= 0.50) or \
                       (label == 'panic' and prob >= 0.65) or \
                       (label == 'sadness' and prob >= 0.75):
                        self.emotion_counts[label] += 1

                # Display the frame in a window
                cv2.imshow("Webcam", frame)

                # Display the results in the text box
                result_str = "감정분석결과:\n"
                for emotion, count in self.emotion_counts.items():
                    result_str += f"{emotion}: {count}\n"
                self.result_text.delete(1.0, tk.END)
                self.result_text.insert(tk.END, result_str)

                if self.result_text_toplevel:
                    self.result_text_toplevel.delete(1.0, tk.END)
                    self.result_text_toplevel.insert(tk.END, result_str)

            self.root.after(250, self.update_webcam)  # Update every 500 ms (0.5 seconds)

    def display_results(self, emotion_counts):
        result_str = "Emotion Analysis Results:\n"
        for emotion, count in emotion_counts.items():
            result_str += f"{emotion}: {count}\n"
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result_str)
        self.label.config(text="캠을 이용한 실시간 감지 혹은 분석하고 싶은 영상을 넣어주세요")

    def update_result_window(self, emotion_counts):
        if self.result_text_toplevel:
            result_str = "Emotion Analysis Results:\n"
            for emotion, count in emotion_counts.items():
                result_str += f"{emotion}: {count}\n"
            self.result_text_toplevel.delete(1.0, tk.END)
            self.result_text_toplevel.insert(tk.END, result_str)

In [8]:
if __name__ == "__main__":
    root = tk.Tk()
    app = EmotionAnalyzerApp(root)
    root.mainloop()

1/1 [==============================] - 0s 29ms/step
